In [1]:
import simple_sim.simulation_objects as Objects
import simple_sim.simulation_server as World
import pybullet_data
import pybullet
import os
import numpy as np
from time import sleep
import simple_sim.simulation_robot as Robots
import simple_sim.simulation_controller as Controls

In [2]:
# Start a simulation server
server = World.simulationServer()

In [4]:
# Add a heightmap
pydata = pybullet_data.getDataPath()
#heightmap = Objects.SimHeightmap(server, './models/heightmap.obj', meshScale = [10, 10, 10])
#plane = pybullet.loadURDF(os.path.join(pydata, 'plane.urdf'))
plane = Objects.SimObject(server, os.path.join(pydata, 'plane.urdf'))

In [5]:
# This loads a relative path
robot = Robots.Robot(server, './models/SimpleWalker/urdf/SimpleWalker.urdf', basePosition = [0, 0, 2])

In [5]:
# Set the gravity
server.setGravity([0, 0, -9.81])

In [6]:
# Simple loop over all joints, not ordered
controller_dict = {}
for joint in sorted(robot.joints.keys()):
    controller_dict.update(
        {joint : Controls.PositionControl(robot, joint, False)}
    )
    #Controls.PositionControl(robot, joint, False)

In [7]:
controller_Group1 = {}
controller_Group2 = {}
controller_keyList = ['Joint_1', 'Joint_2','Joint_3', 'Joint_4']
for controller in controller_keyList:
    controller_Group1.update(
        {controller : controller_dict[controller]}
    )
controller_keyList = ['Joint_13', 'Joint_14','Joint_15', 'Joint_16']
for controller in controller_keyList:
    controller_Group2.update(
        {controller : controller_dict[controller]}
    )

In [8]:
def controlJoints(time, shift, amplitude ,controlGroup):
    counter = 0
    for joint, controller in controlGroup.items():
        if counter == 0:
            reference = amplitude*0.3*np.pi*np.sin(time+shift)
        elif counter == 1:
            reference = amplitude*0.2*np.pi*np.sin(time+shift)
        else:
            reference = 0
        controller.setReference(reference)

In [9]:
# Start the simulation
server.startSimulation()

In [13]:
t = 0
positions = []
orientations = []
times = []
while t < 10:
    # Limit the length of the measurement to 10
    if len(positions) > 10:
        positions = positions[:-1]
    # Get the base position in World frame
    positions.append(pybullet.getBasePositionAndOrientation(robot.id)[0])
    # Get the base orientation in World frame ( Euler Angles)
    orientations.append(
        pybullet.getEulerFromQuaternion(
            # Index 1 == Quaternions
            pybullet.getBasePositionAndOrientation(robot.id)[1]
        )
    )
    # Get the current time
    times.append(t)
    # Apply control law
    controlJoints(t, 0, 2, controller_Group1)
    controlJoints(t, 1, -2, controller_Group2)
    # For slowing down
    sleep(0.2)
    t += 0.2

In [17]:
import pandas as pd

# Convert that to a pandas data frame
pos_data = pd.DataFrame(positions, columns = ['x', 'y', 'z'])
orn_data = pd.DataFrame(orientations,columns = ['x', 'y', 'z'])
# Store as csv file
#pos_data.to_csv('test.csv')
orn_data

,x,y,z
0,-0.015180,0.456390,-0.950843
1,-0.065426,0.467543,-0.946721
2,-0.047869,0.540795,-0.788057
3,0.141871,0.412385,-0.741852
4,0.221859,0.151155,-0.793686
5,0.248634,-0.048059,-0.850266
6,0.220594,-0.209370,-0.881424
7,0.194225,-0.364418,-0.905300
8,0.194266,-0.455099,-0.929127
9,0.209185,-0.503700,-0.946115


In [43]:
pybullet.resetBasePositionAndOrientation(robot.id, [2,2,2], pybullet.getQuaternionFromEuler([0, 0, 0]))

In [41]:
def flowfield(x,y,z):
    return [30*x**2, 20*np.sin(y), 30]

In [37]:
x,y,z = pybullet.getBasePositionAndOrientation(robot.id)[0]
pybullet.applyExternalForce(robot.id, -1, flowfield(x,y,z), [0,0,0], 2 )

In [19]:
t = 0
while t < 10:
    for controller in robot.controller.values():
        controller.setReference(reference = control(t))
    t += 0.1
    sleep(0.5)

NameError: name 'control' is not defined

In [32]:
testy = robot.controller[1]
testy

In [21]:
def slidingMode(joint, reference):
    # Domain of non switching
    eps = 0.2
    error = # Get Joint Pos - reference
        
    if error < -eps:
        return 10
    elif error > eps:
        return -10
    else: 
        return 0